In [3]:
# previous file name:'stripped_down_version'
# convert to 'functions' at 2022.11.9
# now perform as a code directory for development purposes

# containing pure functions:

In [1]:
%%time
def global_pairwise_align(s1:str,s2:str,scoring_matrix='BLOSUM62'):
    global memory
    l1,l2=len(s1),len(s2)
    dp=[[[float('-inf')] for _ in range(l2+1)] for _ in range(l1+1)]
    dp[0][0]=0
    memory={}
    
    assert scoring_matrix in ['BLOSUM62','PAM250'], 'scoring_matrix should be one of BLOSUM62 and PAM250'
    print('scoring matrix is {}'.format(scoring_matrix))
    if scoring_matrix=='BLOSUM62':
        import blosum as bl
        mat = bl.BLOSUM(62) 
        gop,gep=-11,-1
        def get_map_score(a1,a2):
            return mat[a1+a2]
    elif scoring_matrix=='PAM250':
        PAM250 = {
            'A': {'A': 2,  'C': -2, 'D':  0, 'E': 0, 'F': -3, 'G':  1, 'H': -1, 'I': -1, 'K': -1, 'L': -2, 'M': -1, 'N':  0, 'P':  1, 'Q':  0, 'R': -2, 'S':  1, 'T':  1, 'V':  0, 'W': -6, 'Y': -3},
            'C': {'A': -2, 'C': 12, 'D': -5, 'E':-5, 'F': -4, 'G': -3, 'H': -3, 'I': -2, 'K': -5, 'L': -6, 'M': -5, 'N': -4, 'P': -3, 'Q': -5, 'R': -4, 'S':  0, 'T': -2, 'V': -2, 'W': -8, 'Y':  0},
            'D': {'A': 0,  'C': -5, 'D':  4, 'E': 3, 'F': -6, 'G':  1, 'H':  1, 'I': -2, 'K':  0, 'L': -4, 'M': -3, 'N':  2, 'P': -1, 'Q':  2, 'R': -1, 'S':  0, 'T':  0, 'V': -2, 'W': -7, 'Y': -4},
            'E': {'A': 0,  'C': -5, 'D':  3, 'E': 4, 'F': -5, 'G':  0, 'H':  1, 'I': -2, 'K':  0, 'L': -3, 'M': -2, 'N':  1, 'P': -1, 'Q':  2, 'R': -1, 'S':  0, 'T':  0, 'V': -2, 'W': -7, 'Y': -4},
            'F': {'A': -3, 'C': -4, 'D': -6, 'E':-5, 'F':  9, 'G': -5, 'H': -2, 'I':  1, 'K': -5, 'L':  2, 'M':  0, 'N': -3, 'P': -5, 'Q': -5, 'R': -4, 'S': -3, 'T': -3, 'V': -1, 'W':  0, 'Y':  7},
            'G': {'A': 1,  'C': -3, 'D':  1, 'E': 0, 'F': -5, 'G':  5, 'H': -2, 'I': -3, 'K': -2, 'L': -4, 'M': -3, 'N':  0, 'P':  0, 'Q': -1, 'R': -3, 'S':  1, 'T':  0, 'V': -1, 'W': -7, 'Y': -5},
            'H': {'A': -1, 'C': -3, 'D':  1, 'E': 1, 'F': -2, 'G': -2, 'H':  6, 'I': -2, 'K':  0, 'L': -2, 'M': -2, 'N':  2, 'P':  0, 'Q':  3, 'R':  2, 'S': -1, 'T': -1, 'V': -2, 'W': -3, 'Y':  0},
            'I': {'A': -1, 'C': -2, 'D': -2, 'E':-2, 'F':  1, 'G': -3, 'H': -2, 'I':  5, 'K': -2, 'L':  2, 'M':  2, 'N': -2, 'P': -2, 'Q': -2, 'R': -2, 'S': -1, 'T':  0, 'V':  4, 'W': -5, 'Y': -1},
            'K': {'A': -1, 'C': -5, 'D':  0, 'E': 0, 'F': -5, 'G': -2, 'H':  0, 'I': -2, 'K':  5, 'L': -3, 'M':  0, 'N':  1, 'P': -1, 'Q':  1, 'R':  3, 'S':  0, 'T':  0, 'V': -2, 'W': -3, 'Y': -4},
            'L': {'A': -2, 'C': -6, 'D': -4, 'E':-3, 'F':  2, 'G': -4, 'H': -2, 'I':  2, 'K': -3, 'L':  6, 'M':  4, 'N': -3, 'P': -3, 'Q': -2, 'R': -3, 'S': -3, 'T': -2, 'V':  2, 'W': -2, 'Y': -1},
            'M': {'A': -1, 'C': -5, 'D': -3, 'E':-2, 'F':  0, 'G': -3, 'H': -2, 'I':  2, 'K':  0, 'L':  4, 'M':  6, 'N': -2, 'P': -2, 'Q': -1, 'R':  0, 'S': -2, 'T': -1, 'V':  2, 'W': -4, 'Y': -2},
            'N': {'A': 0,  'C': -4, 'D':  2, 'E': 1, 'F': -3, 'G':  0, 'H':  2, 'I': -2, 'K':  1, 'L': -3, 'M': -2, 'N':  2, 'P':  0, 'Q':  1, 'R':  0, 'S':  1, 'T':  0, 'V': -2, 'W': -4, 'Y': -2},
            'P': {'A': 1,  'C': -3, 'D': -1, 'E':-1, 'F': -5, 'G':  0, 'H':  0, 'I': -2, 'K': -1, 'L': -3, 'M': -2, 'N':  0, 'P':  6, 'Q':  0, 'R':  0, 'S':  1, 'T':  0, 'V': -1, 'W': -6, 'Y': -5},
            'Q': {'A': 0,  'C': -5, 'D':  2, 'E': 2, 'F': -5, 'G': -1, 'H':  3, 'I': -2, 'K':  1, 'L': -2, 'M': -1, 'N':  1, 'P':  0, 'Q':  4, 'R':  1, 'S': -1, 'T': -1, 'V': -2, 'W': -5, 'Y': -4},
            'R': {'A': -2, 'C': -4, 'D': -1, 'E':-1, 'F': -4, 'G': -3, 'H':  2, 'I': -2, 'K':  3, 'L': -3, 'M':  0, 'N':  0, 'P':  0, 'Q':  1, 'R':  6, 'S':  0, 'T': -1, 'V': -2, 'W':  2, 'Y': -4},
            'S': {'A': 1,  'C':  0, 'D':  0, 'E': 0, 'F': -3, 'G':  1, 'H': -1, 'I': -1, 'K':  0, 'L': -3, 'M': -2, 'N':  1, 'P':  1, 'Q': -1, 'R':  0, 'S':  2, 'T':  1, 'V': -1, 'W': -2, 'Y': -3},
            'T': {'A': 1,  'C': -2, 'D':  0, 'E': 0, 'F': -3, 'G':  0, 'H': -1, 'I':  0, 'K':  0, 'L': -2, 'M': -1, 'N':  0, 'P':  0, 'Q': -1, 'R': -1, 'S':  1, 'T':  3, 'V':  0, 'W': -5, 'Y': -3},
            'V': {'A': 0,  'C': -2, 'D': -2, 'E':-2, 'F': -1, 'G': -1, 'H': -2, 'I':  4, 'K': -2, 'L':  2, 'M':  2, 'N': -2, 'P': -1, 'Q': -2, 'R': -2, 'S': -1, 'T':  0, 'V':  4, 'W': -6, 'Y': -2},
            'W': {'A': -6, 'C': -8, 'D': -7, 'E':-7, 'F':  0, 'G': -7, 'H': -3, 'I': -5, 'K': -3, 'L': -2, 'M': -4, 'N': -4, 'P': -6, 'Q': -5, 'R':  2, 'S': -2, 'T': -5, 'V': -6, 'W': 17, 'Y':  0},
            'Y': {'A': -3, 'C':  0, 'D': -4, 'E':-4, 'F':  7, 'G': -5, 'H':  0, 'I': -1, 'K': -4, 'L': -1, 'M': -2, 'N': -2, 'P': -5, 'Q': -4, 'R': -4, 'S': -3, 'T': -3, 'V': -2, 'W':  0, 'Y': 10},
        }
        gop,gep=-12,-4
        def get_map_score(a1,a2):
            return PAM250[a1][a2]
        
    def get_score(m,n,pm,pn):
        score=0
        if pm==m-1 and pn==n-1:
            score=get_map_score(s1[pm],s2[pn])
        elif pm==m:
            score=gop if (not (pm,pn) in memory) or memory[(pm,pn)][0]!=m else gep
        elif pn==n:
            score=gop if (not (pm,pn) in memory) or memory[(pm,pn)][1]!=n else gep
        return dp[pm][pn]+score
    
    def get_possible_previous_position(i,j):
        if i==0:
            return [(i,j-1)]
        elif j==0:
            return [(i-1,j)]
        else:
            return [(i-1,j-1),(i-1,j),(i,j-1)]
    
    def trace_alignment(dp,memory,s1,s2):
        l1,l2=len(dp),len(dp[0])
        rs1,rs2='',''
        current=(l1-1,l2-1)
        last=memory[current]
        while last != (0,0):
            last=memory[current]
            cm,cn=current
            lm,ln=last
            rs1=rs1+'-' if lm==cm else rs1+s1[current[0]-1]
            rs2=rs2+'-' if ln==cn else rs2+s2[current[1]-1]
            current=last
        print('\033[91m>>>alignment result\033[0m')
        print(rs1[::-1])
        print(rs2[::-1])
    
    all_position=(lambda x,y :[(z0,z1) for z0 in range(x) for z1 in range(y)])(l1+1,l2+1)[1:]
    for m,n in all_position:
        previous=get_possible_previous_position(m,n)
        choice=[]
        for pm,pn in previous:
            score=get_score(m,n,pm,pn)
            choice.append((score,(pm,pn)))
        choice.sort(key=lambda tup: tup[0], reverse=True)
        lm,ln=choice[0][1][0],choice[0][1][1]
        dp[m][n]=choice[0][0]
        memory[(m,n)]=choice[0][1]

    print('\033[91m>>>global optimal score:',dp[-1][-1],'\033[0m')
    trace_alignment(dp,memory,s1,s2)
    return dp, memory

test_case={
's1':'YYKNMWIRVMSKWVSSWYRFFLPWCYYPSVCKCEFPCTMLGEEQGIANLGDKISPKNKSLINNHDQMTDNLCKYWVKNGWDM',
's2':'YYKNMLPWCYYPVFPCTMLGEEQGIAPPIGLKISPKNKSYDHMINNHDQETDNCCKYWVKNGYDRYPP'
}

dp,memory=global_pairwise_align(**test_case)
print('\033[91m>>>time report\033[0m')

scoring matrix is BLOSUM62
>>>global optimal score: 220.0 
>>>alignment result
YYKNMWIRVMSKWVSSWYRFFLPWCYYPSVCKCEFPCTMLGEEQGIA-NLGDKISPKNKS---LINNHDQMTDNLCKYWVKNGWD---M
YYKN----------------MLPWCYYP-----VFPCTMLGEEQGIAPPIGLKISPKNKSYDHMINNHDQETDNCCKYWVKNGYDRYPP
>>>time report
CPU times: user 24.2 ms, sys: 2.05 ms, total: 26.2 ms
Wall time: 25.2 ms


In [2]:
def test():
    print('import function test')